In [1]:
import os
import sys
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import math
import scipy.stats as stats
import torch.optim as optim
from torchsummary import summary
from tqdm.notebook import tqdm_notebook


import import_ipynb
from Trainer import Trainer
from CustomDataset import ControlsDataset
from Models import RegConvNet, ClassConvNet, ClassConvNetNorm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()
print('using device', device)

importing Jupyter notebook from Trainer.ipynb
importing Jupyter notebook from CustomDataset.ipynb
importing Jupyter notebook from Models.ipynb
torch.Size([2, 64, 20, 43])
torch.Size([2, 55040])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 24, 108, 198]           1,824
              ReLU-2         [-1, 24, 108, 198]               0
            Conv2d-3           [-1, 36, 52, 97]          21,636
              ReLU-4           [-1, 36, 52, 97]               0
            Conv2d-5           [-1, 48, 24, 47]          43,248
              ReLU-6           [-1, 48, 24, 47]               0
            Conv2d-7           [-1, 64, 22, 45]          27,712
              ReLU-8           [-1, 64, 22, 45]               0
            Conv2d-9           [-1, 64, 20, 43]          36,928
          Dropout-10           [-1, 64, 20, 43]               0
           Linear-11                 

In [50]:
dataset = ControlsDataset(img_folder='../data/cropped/', csv_path='../data/cropped/data.csv')
dataset.make_dataloaders()

dataset.labels.num_categories = 14
dataset.labels.transform.categorize(1.05, -1.05, 14)

Total training stacks 69
Total validation stacks 18


[[-1.05, -0.9],
 [-0.9, -0.75],
 [-0.75, -0.6],
 [-0.6, -0.45],
 [-0.45, -0.3],
 [-0.3, -0.15],
 [-0.15, 0.0],
 [0.0, 0.15],
 [0.15, 0.3],
 [0.3, 0.45],
 [0.45, 0.6],
 [0.6, 0.75],
 [0.75, 0.9],
 [0.9, 1.05]]

In [51]:
# import torchvision.models as models
# resnet50 = models.resnet50(pretrained=True)
# for param in resnet50.parameters():
#     param.requires_grad = False
# resnet50.fc = nn.Sequential(nn.Linear(2048, 1))
# resnet50 = resnet50.to(device)
# # print(resnet50)
model = ClassConvNet(3, 14).to(device)
summary(model, (3, 220, 400))

epochs = 30
report_period = 10
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

torch.Size([2, 64, 20, 43])
torch.Size([2, 55040])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 24, 108, 198]           1,824
              ReLU-2         [-1, 24, 108, 198]               0
            Conv2d-3           [-1, 36, 52, 97]          21,636
              ReLU-4           [-1, 36, 52, 97]               0
            Conv2d-5           [-1, 48, 24, 47]          43,248
              ReLU-6           [-1, 48, 24, 47]               0
            Conv2d-7           [-1, 64, 22, 45]          27,712
              ReLU-8           [-1, 64, 22, 45]               0
            Conv2d-9           [-1, 64, 20, 43]          36,928
          Dropout-10           [-1, 64, 20, 43]               0
           Linear-11                  [-1, 100]       5,504,100
             ReLU-12                  [-1, 100]               0
           Linear-13                   [-1, 50]     

In [52]:
trainer = Trainer(device, model, dataset, optimizer, criterion)
trainer.fit(epochs, report_period)

RuntimeError: CUDA out of memory. Tried to allocate 258.00 MiB (GPU 0; 7.93 GiB total capacity; 5.25 GiB already allocated; 49.06 MiB free; 107.27 MiB cached)

In [ ]:
# from tqdm.notebook import tqdm_notebook
# resnet50.train()
# iter_no = 0
# report_period = 20
# batch_size = 20

# for epoch in range(epochs):
#     percent = 0
#     for i_batch, sampled_batch in tqdm_notebook(enumerate(dataset.dataloader),
#                                                 total=len(dataset.dataloader),
#                                                 desc= str(percent)):
        
#         images = sampled_batch['image'].to(device).float()
#         controls = sampled_batch['control'].to(device).float()
#         # multiply on controls by 100
#         controls *= 100
        
#         # forward pass
#         prediction = resnet50(images)
#         loss = criterion(prediction, controls)
#         loss_item = loss.item()
        
#         #backwards pass
#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()
#         print(loss.item())

#         iter_no += 1
#         if iter_no % report_period == 0:
#             print("saved to tensorboard")
        

#     out = "{0},{1}\tLoss:{2}\tAllocated:{3}GB\tCached:{4}GB\n"
#     print(out.format(str(epoch),
#                     str(iter_no),
#                     round(loss.item(),5),
#                      'na', 'na'
#                      #round(torch.cuda.memory_allocated(0)/1024**3,3),
#                      #round(torch.cuda.memory_allocated(0)/1024**3,3)
#                     ))


In [ ]:
def save(epoch,model,path):
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict()},
            path)

In [ ]:
save(30,resnet50,"resnet.pt")